In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, DebertaTokenizer, DebertaForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
from urllib import request
import pandas as pd
import logging
import torch
from collections import Counter
from ast import literal_eval
from sklearn.metrics import f1_score
from dont_patronize_me import DontPatronizeMe
import nlpaug.augmenter.word as naw

2024-03-01 20:15:30.775139: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-01 20:15:30.775208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-01 20:15:30.776212: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-01 20:15:30.782859: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-01 20:15:32.741964: W tensorflow/compiler/tf2

In [2]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [3]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')
			
dpm = DontPatronizeMe('./Data', './Data')
dpm.load_task1()

trids = pd.read_csv('Data/train_semeval_parids-labels.csv')
teids = pd.read_csv('Data/dev_semeval_parids-labels.csv')

trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)

data=dpm.train_task1_df

rows = [] 
for idx in range(len(trids)):  
  parid = trids.par_id[idx]
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })

trdf1 = pd.DataFrame(rows)

rows = []
for idx in range(len(teids)):  
  parid = teids.par_id[idx]
  keyword = data.loc[data.par_id == parid].keyword.values[0]
  text = data.loc[data.par_id == parid].text.values[0]
  label = data.loc[data.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'community':keyword,
      'text':text,
      'label':label
  })
  
tedf1 = pd.DataFrame(rows)

## downsampling
# pcldf = trdf1[trdf1.label==1]
# npos = len(pcldf)

# training_set1 = pd.concat([pcldf,trdf1[trdf1.label==0][:npos*2]])

## normal
training_set1 = trdf1

device = "cuda:0" if torch.cuda.is_available() else "cpu"

print(len(training_set1))
torch.cuda.empty_cache()

8375


In [4]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# model = RobertaForSequenceClassification.from_pretrained('roberta-base')

tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base')

class PatronizingDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        return {'text': text, 'label': label}

train_texts, val_texts, train_labels, val_labels = train_test_split(training_set1['text'].to_numpy(), training_set1['label'].to_numpy(), test_size=0.2, random_state=42)
print('val size', len(val_texts))

train_dataset = PatronizingDataset(train_texts, train_labels)
val_dataset = PatronizingDataset(val_texts, val_labels)

train_loader = DataLoader(train_dataset, batch_size=5, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=5, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    n = len(train_loader)
    cnt = 0
    for batch in train_loader:

        print(cnt/n)
        cnt += 1

        inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors='pt')
        inputs = {key: tensor.to(device) for key, tensor in inputs.items()}
        labels = batch['label'].to(device)
        optimizer.zero_grad()
        outputs = model(**inputs, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        for key in inputs:
            inputs[key] = inputs[key].cpu()
        labels = labels.cpu()
        torch.cuda.empty_cache()

    avg_train_loss = total_loss / len(train_loader)
    
    model.eval()
    val_preds = []
    val_true = []
    for batch in val_loader:
        inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors='pt')
        inputs = {key: tensor.to(device) for key, tensor in inputs.items()}
        labels = batch['label'].to(device)
        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).tolist()
        val_preds.extend(preds)
        val_true.extend(labels.tolist())
    val_accuracy = accuracy_score(val_true, val_preds)
    print(f'Epoch {epoch+1}/{num_epochs}, Avg. Train Loss: {avg_train_loss:.4f}, f1 score: {f1_score(val_true, val_preds)}, Val. Accuracy: {val_accuracy:.4f}')
    torch.cuda.empty_cache()

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


val size 1675


/vol/bitbucket/kh123/nlp_cw/venv/lib/python3.10/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0.0
0.0007462686567164179
0.0014925373134328358
0.002238805970149254
0.0029850746268656717
0.0037313432835820895
0.004477611940298508
0.0052238805970149255
0.005970149253731343
0.006716417910447761
0.007462686567164179
0.008208955223880597
0.008955223880597015
0.009701492537313432
0.010447761194029851
0.011194029850746268
0.011940298507462687
0.012686567164179104
0.013432835820895522
0.014179104477611941
0.014925373134328358
0.015671641791044775
0.016417910447761194
0.017164179104477612
0.01791044776119403
0.018656716417910446
0.019402985074626865
0.020149253731343283
0.020895522388059702
0.02164179104477612
0.022388059701492536
0.023134328358208955
0.023880597014925373
0.024626865671641792
0.025373134328358207
0.026119402985074626
0.026865671641791045
0.027611940298507463
0.028358208955223882
0.029104477611940297
0.029850746268656716
0.030597014925373135
0.03134328358208955
0.03208955223880597
0.03283582089552239
0.033582089552238806
0.034328358208955224
0.03507462686567164
0.03582089

In [5]:
dev_dataset = PatronizingDataset(tedf1['text'].to_list(), tedf1['label'].to_list())
dev_loader = DataLoader(dev_dataset, batch_size=5, shuffle=True)

model.eval()
dev_preds = []
dev_true = []
n = len(dev_loader)
cnt = 0
for batch in dev_loader:
    print(cnt/n)
    cnt += 1
    inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors='pt')
    inputs = {key: tensor.to(device) for key, tensor in inputs.items()}
    labels = batch['label'].to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    preds = torch.argmax(logits, dim=1).tolist()
    dev_preds.extend(preds)
    dev_true.extend(labels.tolist())

    for key in inputs:
        inputs[key] = inputs[key].cpu()
    labels = labels.cpu()
    torch.cuda.empty_cache()

dev_accuracy = accuracy_score(dev_true, dev_preds)
print('f1 score', f1_score(dev_true, dev_preds))
print('dev accruacy', dev_accuracy)

0.0
0.002386634844868735
0.00477326968973747
0.007159904534606206
0.00954653937947494
0.011933174224343675
0.014319809069212411
0.016706443914081145
0.01909307875894988
0.021479713603818614
0.02386634844868735
0.026252983293556086
0.028639618138424822
0.031026252983293555
0.03341288782816229
0.03579952267303103
0.03818615751789976
0.0405727923627685
0.04295942720763723
0.045346062052505964
0.0477326968973747
0.050119331742243436
0.05250596658711217
0.05489260143198091
0.057279236276849645
0.059665871121718374
0.06205250596658711
0.06443914081145585
0.06682577565632458
0.06921241050119331
0.07159904534606205
0.07398568019093078
0.07637231503579953
0.07875894988066826
0.081145584725537
0.08353221957040573
0.08591885441527446
0.0883054892601432
0.09069212410501193
0.09307875894988067
0.0954653937947494
0.09785202863961814
0.10023866348448687
0.1026252983293556
0.10501193317422435
0.10739856801909307
0.10978520286396182
0.11217183770883055
0.11455847255369929
0.11694510739856802
0.11933174